# **DEEP LEARNING**

**UNIVERSIDAD MILITAR NUEVA GRANADA**

**Deep learning - Fine tuning con selección de capas**

Ingeniería en Telecomunicaciones

© Material de uso exclusivo de la asignatura Deep learning (Docente Diego Renza Torres)

In [ ]:
!nvidia-smi

Fri Apr 30 12:19:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy

import pathlib

# Dataset

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
print(os.getcwd())

Mounted at /content/drive
/content


In [ ]:
!unzip -qq "/content/drive/My Drive/Data_CV/Cats_dogs_merged.zip" -d /content/

In [ ]:
data_dir = "/content/Cats_dogs_merged"
data_dir = pathlib.Path(data_dir)

In [ ]:
# Dimensiones de imagen y tamaño de batch
img_height = 224
img_width = 224

In [ ]:
# Data splitting (entrenamiento y validación)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=24
  )

Found 3000 files belonging to 2 classes.
Using 2400 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=24
  )

Found 3000 files belonging to 2 classes.
Using 600 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['Cats', 'Dogs']


In [ ]:
# Dimensiones de los tensores
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(24, 224, 224, 3)
(24,)


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Modelo

In [ ]:
# Modelo pre-entrenado
base_model = VGG16(weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
# Agregar una capa de average pooling
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Agregar una capa Fully Connected
x = Dense(1024, activation='relu')(x)

# Agregar una capa FC del número de clases, por ejemplo 2 (Cats & Dogs)
predictions = Dense(2, activation='softmax')(x)

# Modelo final para entrenar
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [ ]:
# Visualizar los nombres de las capas y los índices de las capas
# para determinar cuáles capas congelar:

for i, layer in enumerate(base_model.layers):
  print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool


# Entrenamiento

In [ ]:
# Definir las capas que NO se entrenarán (False)
# Definir las capas que SÍ se entrenarán (True)

for layer in model.layers[:15]:
  layer.trainable = False
for layer in model.layers[15:]:
  layer.trainable = True

In [ ]:
# Compilar el modelo (la compilación debe realizarse después de definir las capas como "non-trainable")
#loss=SparseCategoricalCrossentropy(from_logits=True),
model.compile(
    optimizer='rmsprop',
     loss=SparseCategoricalCrossentropy(),
     metrics=['accuracy']
    )

In [ ]:
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=5,
                    verbose=1,
                    #callbacks=callbacks_list
                    )

Epoch 1/5
100/100 [==============================] - 71s 326ms/step - loss: 274.3194 - accuracy: 0.5050 - val_loss: 0.6931 - val_accuracy: 0.5067
Epoch 2/5
100/100 [==============================] - 32s 315ms/step - loss: 1.3240 - accuracy: 0.4938 - val_loss: 0.6931 - val_accuracy: 0.5067
Epoch 3/5
100/100 [==============================] - 31s 315ms/step - loss: 0.6937 - accuracy: 0.5008 - val_loss: 0.6932 - val_accuracy: 0.5067
Epoch 4/5
100/100 [==============================] - 31s 314ms/step - loss: 0.6938 - accuracy: 0.4950 - val_loss: 0.6931 - val_accuracy: 0.5067
Epoch 5/5
100/100 [==============================] - 31s 315ms/step - loss: 0.6937 - accuracy: 0.4925 - val_loss: 0.6931 - val_accuracy: 0.5067


In [ ]:
img_path = 'Perro5.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)

# Predecir la clase
result = np.argmax(preds, axis=-1)         # Multiclass (softmax)
print(class_names[result[0]])


Dogs
